In [1]:
# import library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim

In [2]:
data = pd.read_excel('data/all_states_all_years.xlsx')
data.head()

,Year,Date_Temp,Date,State,Region,Place,Name,Line Check
0,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak, Johor",1
1,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Sialang Dalam,"Kg. Sialang Dalam, Tangkak, Johor",1
2,2015.0,3/1,3/1/2015,Johor,Kota Tinggi,Kg. Lepau,"Kg. Lepau, Kota Tinggi, Johor",1
3,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seri Makmor,"Kg. Seri Makmor, Tangkak, Johor",1
4,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak, Johor",1


In [3]:
# testing geolocator library
geolocator = Nominatim(user_agent="u2004763@siswa.um.edu.my")
location = geolocator.geocode("Johor Bahru") # testing

print(location.point)
print(location.address)

# print out longitude and latitude
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

1 30m 12.798s N, 103 44m 58.411s E
Johor Bahru, Iskandar Malaysia, Johor, 80350, Malaysia
Latitude: 1.503555
Longitude: 103.7495586


In [4]:
# geocoding function
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None, None

data[['Latitude', 'Longitude']] = data.apply(lambda x: my_geocoder(x['Name']), axis=1)

# check the percentage of data successfully geocoded
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

ValueError: Columns must be same length as key

In [ ]:
# Drop location that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))
data.crs = {'init': 'epsg:4326'}
data.head()

In [ ]:
# export csv
data.to_csv('data/all_states_all_years_geocoded.csv', index=False)